In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import seaborn as sn
from sklearn.metrics import confusion_matrix , classification_report

Load the data



In [ ]:
df = pd.read_csv('/content/drive/MyDrive/DeepLearning/explore/telco.csv')

First of all, drop customerID column as it is of no use



In [ ]:
df = df.drop(columns='customerID')
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.groupby(['MultipleLines']).agg('count')

some values seems to be not numbers but blank string

In [ ]:
df.TotalCharges[df.TotalCharges.values == ' ']

Remove rows with space in TotalCharges



In [ ]:
df[pd.to_numeric(df.TotalCharges,errors='coerce').isnull()]
df1 = df[df.TotalCharges!=' ']
df1.shape

In [ ]:
df1.TotalCharges = pd.to_numeric(df1.TotalCharges)

In [ ]:
df1.TotalCharges.dtypes

Data Visualization

In [ ]:
tenure_churn_no = df1[df1.Churn=='No'].tenure
tenure_churn_yes = df1[df1.Churn=='Yes'].tenure

plt.xlabel("tenure")
plt.ylabel("Number Of Customers")
plt.title("Customer Churn Prediction Visualiztion")

blood_sugar_men = [113, 85, 90, 150, 149, 88, 93, 115, 135, 80, 77, 82, 129]
blood_sugar_women = [67, 98, 89, 120, 133, 150, 84, 69, 89, 79, 120, 112, 100]

plt.hist([tenure_churn_yes, tenure_churn_no], rwidth=0.95, color=['green','red'],label=['Churn=Yes','Churn=No'])
plt.legend()

In [ ]:
tenure_churn_no = df1[df1.Churn=='No'].MonthlyCharges
tenure_churn_yes = df1[df1.Churn=='Yes'].MonthlyCharges

plt.xlabel("MonthlyCharges")
plt.ylabel("Number Of Customers")
plt.title("Customer Churn Prediction Visualiztion")

blood_sugar_men = [113, 85, 90, 150, 149, 88, 93, 115, 135, 80, 77, 82, 129]
blood_sugar_women = [67, 98, 89, 120, 133, 150, 84, 69, 89, 79, 120, 112, 100]

plt.hist([tenure_churn_yes, tenure_churn_no], rwidth=0.95, color=['green','red'],label=['Churn=Yes','Churn=No'])
plt.legend()

In [ ]:
tenure_churn_no = df1[df1.Churn=='No'].SeniorCitizen
tenure_churn_yes = df1[df1.Churn=='Yes'].SeniorCitizen

plt.xlabel("SeniorCitizen")
plt.ylabel("Number Of Customers")
plt.title("Customer Churn Prediction Visualiztion")

blood_sugar_men = [113, 85, 90, 150, 149, 88, 93, 115, 135, 80, 77, 82, 129]
blood_sugar_women = [67, 98, 89, 120, 133, 150, 84, 69, 89, 79, 120, 112, 100]

plt.hist([tenure_churn_yes, tenure_churn_no], rwidth=0.95, color=['green','red'],label=['Churn=Yes','Churn=No'])
plt.legend()

Many of the columns are yes, no etc. Let's print unique values in object columns to see data values

In [ ]:
def print_unique_col(df):
  for columns in df:
    if df[columns].dtype == 'object':
      print(columns,'-', df[columns].unique())

In [ ]:
print_unique_col(df1)

In [ ]:
df1 = df1.replace('No internet service','No')
df1 = df1.replace('No phone service','No')

In [ ]:
print_unique_col(df1)

Convert Categorical Data into numerical data 

In [ ]:
df1 = df1.replace({'No':0, 'Yes':1, 'Male':0, 'Female':1})
df1['InternetService']  = df1['InternetService'].replace(0,'No')
print_unique_col(df1)

One hot encoding for categorical columns

In [ ]:
df2 = pd.get_dummies(data=df1, columns=['InternetService','Contract','PaymentMethod'])
df2.head()

In [ ]:
df2.tenure.describe()

In [ ]:
col_to_scale = ['tenure','MonthlyCharges','TotalCharges']
scaler = MinMaxScaler()
df2[col_to_scale] = scaler.fit_transform(df2[col_to_scale])
df2.head()

In [ ]:
x = df2.drop(columns = 'Churn')
y = df2['Churn']

Train test split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 5) 

In [ ]:
len(x_test)

In [ ]:
x_train.shape

Build a model (ANN) in tensorflow/keras



In [ ]:
model = keras.Sequential([
        keras.layers.Dense(20,input_shape = (26,), activation = 'relu'),
        keras.layers.Dense(1, activation = 'sigmoid'),
])

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

model.fit(x_train,y_train,epochs = 100)

In [ ]:
input = x_test[0:5]
model.predict(input)

In [ ]:
yp = y_test

In [ ]:
y_pred = []
for element in yp:
    if element > 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [ ]:
y_pred

In [ ]:
yp

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
cm = tf.math.confusion_matrix(labels=y_test,predictions=y_pred)
plt.figure(figsize = (10,7))
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')